In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from pytorchcv.model_provider import get_model

In [2]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [3]:
cache = "/scratch/doa240/.cache/datasets"
train_data = torchvision.datasets.CIFAR10(root=cache, train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(train_data, batch_size=4, shuffle=True, num_workers=4)

Files already downloaded and verified


In [4]:
classes = ('Airplane', 'Car', 'Bird', 'Cat', 'Deer', 'Dog', 'Frog', 'Horse', 'Ship', 'Truck')

In [10]:
model = get_model("zfnet", pretrained=False, root= "/scratch/doa240/.torch/models")
model.eval()

AlexNet(
  (features): Sequential(
    (stage1): Sequential(
      (unit1): AlexConv(
        (conv): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2), padding=(1, 1))
        (activ): ReLU(inplace=True)
      )
      (pool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    )
    (stage2): Sequential(
      (unit1): AlexConv(
        (conv): Conv2d(96, 256, kernel_size=(5, 5), stride=(2, 2))
        (activ): ReLU(inplace=True)
      )
      (pool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    )
    (stage3): Sequential(
      (unit1): AlexConv(
        (conv): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (activ): ReLU(inplace=True)
      )
      (unit2): AlexConv(
        (conv): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (activ): ReLU(inplace=True)
      )
      (unit3): AlexConv(
        (conv): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 

In [11]:
import torch.nn as nn
model.output.fc2.fc = nn.Linear(4096,1024)
model.output.fc3 = nn.Linear(1024,10)
model.eval()

AlexNet(
  (features): Sequential(
    (stage1): Sequential(
      (unit1): AlexConv(
        (conv): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2), padding=(1, 1))
        (activ): ReLU(inplace=True)
      )
      (pool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    )
    (stage2): Sequential(
      (unit1): AlexConv(
        (conv): Conv2d(96, 256, kernel_size=(5, 5), stride=(2, 2))
        (activ): ReLU(inplace=True)
      )
      (pool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    )
    (stage3): Sequential(
      (unit1): AlexConv(
        (conv): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (activ): ReLU(inplace=True)
      )
      (unit2): AlexConv(
        (conv): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (activ): ReLU(inplace=True)
      )
      (unit3): AlexConv(
        (conv): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 

In [12]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [13]:
model.to(device)

AlexNet(
  (features): Sequential(
    (stage1): Sequential(
      (unit1): AlexConv(
        (conv): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2), padding=(1, 1))
        (activ): ReLU(inplace=True)
      )
      (pool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    )
    (stage2): Sequential(
      (unit1): AlexConv(
        (conv): Conv2d(96, 256, kernel_size=(5, 5), stride=(2, 2))
        (activ): ReLU(inplace=True)
      )
      (pool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    )
    (stage3): Sequential(
      (unit1): AlexConv(
        (conv): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (activ): ReLU(inplace=True)
      )
      (unit2): AlexConv(
        (conv): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (activ): ReLU(inplace=True)
      )
      (unit3): AlexConv(
        (conv): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 

In [14]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [15]:
import time

In [16]:
for epoch in range(7):  # loop over the dataset multiple times

    running_loss = 0.0
    start_time = time.time()
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        output = model(inputs)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()

        #Time
        end_time = time.time()
        time_taken = end_time - start_time

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
            print('Time:',time_taken)
            running_loss = 0.0

print('Finished Training of AlexNet')

[1,  2000] loss: 1.136
Time: 29.762009382247925
[1,  4000] loss: 0.792
Time: 51.84672021865845
[1,  6000] loss: 0.706
Time: 73.92991209030151
[1,  8000] loss: 0.666
Time: 95.86893558502197
[1, 10000] loss: 0.601
Time: 117.68399667739868
[1, 12000] loss: 0.550
Time: 139.4816460609436
[2,  2000] loss: 0.405
Time: 21.83594584465027
[2,  4000] loss: 0.425
Time: 53.01881694793701
[2,  6000] loss: 0.417
Time: 91.6776351928711
[2,  8000] loss: 0.420
Time: 130.3037703037262
[2, 10000] loss: 0.418
Time: 168.922465801239
[2, 12000] loss: 0.391
Time: 207.55753421783447
[3,  2000] loss: 0.249
Time: 38.182865381240845
[3,  4000] loss: 0.292
Time: 75.82924032211304
[3,  6000] loss: 0.291
Time: 113.60785818099976
[3,  8000] loss: 0.260
Time: 152.18091821670532
[3, 10000] loss: 0.288
Time: 190.738862991333
[3, 12000] loss: 0.264
Time: 229.3140938282013
[4,  2000] loss: 0.148
Time: 38.226070404052734
[4,  4000] loss: 0.177
Time: 76.44042015075684
[4,  6000] loss: 0.187
Time: 115.13539218902588
[4,  800

In [17]:
test_data = torchvision.datasets.CIFAR10(root=cache, train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(test_data, batch_size=4, shuffle=False, num_workers=4)

Files already downloaded and verified


In [18]:
#Testing Accuracy
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %.2f %%' % (100 * correct / total))

Accuracy of the network on the 10000 test images: 85.37 %


In [19]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of Airplane : 92 %
Accuracy of   Car : 95 %
Accuracy of  Bird : 87 %
Accuracy of   Cat : 64 %
Accuracy of  Deer : 89 %
Accuracy of   Dog : 77 %
Accuracy of  Frog : 92 %
Accuracy of Horse : 88 %
Accuracy of  Ship : 82 %
Accuracy of Truck : 83 %


In [20]:
# #Verifying average accuracy of the network
# avg = 0
# for i in range(10):
#   temp = (100 * class_correct[i] / class_total[i])
#   avg = avg + temp
# avg = avg/10
# print('Average accuracy = ', avg)  